# Adversarial Autoencoder

ref. 

![architecture](arch.jpg)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
# from keras.layers import merge
from keras.layers import Lambda, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

Using TensorFlow backend.


# Encoder

In [3]:
def build_encoder(img_shape, latent_dim):
    # Encoder

    img = Input(shape=img_shape)

    h = Flatten()(img)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    mu = Dense(latent_dim)(h)
    log_var = Dense(latent_dim)(h)
    
    # NOTE: original code uses keras.layer.merge() function which is deprecated
    #     latent_repr = merge([mu, log_var],
    #                          mode=lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2),
    #                          output_shape=lambda p: p[0])
    
    # NOW: we use Lambda layer to do the same thing
    latent_repr = Lambda(lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2), 
                         output_shape=lambda p: p[0])([mu, log_var])   
    
    return Model(img, latent_repr)

# Generator

In [4]:
def build_decoder(img_shape, latent_dim):

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    z = Input(shape=(latent_dim,))
    img = model(z)

    return Model(z, img)

# Discriminator

In [5]:
def build_discriminator(latent_dim):

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.summary()

    encoded_repr = Input(shape=(latent_dim, ))
    validity = model(encoded_repr)

    return Model(encoded_repr, validity)

# Save images

In [6]:
def sample_images(epoch, decoder, latent_dim):
    r, c = 5, 5

    z = np.random.normal(size=(r*c, latent_dim))
    gen_imgs = decoder.predict(z)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

# Save model

In [7]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

def save_model(G, D):
    save(G, "aae_generator")
    save(D, "aae_discriminator")

# Train the GAN

In [8]:
def train(encoder, decoder, adversarial_autoencoder, D, latent_dim, epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # NOTE: isn't that weird that encodeded imgs are latent fake ???
        latent_fake = encoder.predict(imgs)
        # latent_fake : from random
        latent_real = np.random.normal(size=(batch_size, latent_dim))

        # Train the discriminator
        d_loss_real = D.train_on_batch(latent_real, valid)
        d_loss_fake = D.train_on_batch(latent_fake, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Adversarial autoencoder
        # ---------------------

        # Train the combined stack
        g_loss = adversarial_autoencoder.train_on_batch(imgs, [imgs, valid])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(epoch, decoder, latent_dim)

# main()

In [9]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 10

In [10]:
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Build and compile the discriminator
D = build_discriminator(latent_dim)
D.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 137,217
Trainable params: 137,217
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Build the encoder / decoder
encoder = build_encoder(img_shape, latent_dim)
decoder = build_decoder(img_shape, latent_dim)

img = Input(shape=img_shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 784)               402192    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
Total params: 670,480
Trainable params: 670,480
Non-trainable params: 0
________________________________________________

In [13]:
# The generator takes the image, encodes it and reconstructs it
# from the encoding
encoded_repr = encoder(img)
reconstructed_img = decoder(encoded_repr)

In [14]:
# For the adversarial_autoencoder model we will only train the generator
D.trainable = False

In [15]:
# The discriminator determines validity of the encoding
validity = D(encoded_repr)

In [16]:
# The adversarial_autoencoder model  (stacked generator and discriminator)
adversarial_autoencoder = Model(img, [reconstructed_img, validity])
adversarial_autoencoder.compile(loss=['mse', 'binary_crossentropy'],
    loss_weights=[0.999, 0.001],
    optimizer=optimizer)

In [17]:
epochs=20000
train(encoder, decoder, adversarial_autoencoder, D, 
      latent_dim, epochs=epochs, batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.716873, acc: 23.44%] [G loss: 0.941897, mse: 0.942123]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.680778, acc: 56.25%] [G loss: 0.920174, mse: 0.920355]
2 [D loss: 0.648970, acc: 75.00%] [G loss: 0.898855, mse: 0.898928]
3 [D loss: 0.576418, acc: 75.00%] [G loss: 0.834570, mse: 0.834184]
4 [D loss: 0.420752, acc: 82.81%] [G loss: 0.750212, mse: 0.748913]
5 [D loss: 0.332738, acc: 84.38%] [G loss: 0.637641, mse: 0.634767]
6 [D loss: 0.318552, acc: 85.94%] [G loss: 0.508700, mse: 0.504168]
7 [D loss: 0.289780, acc: 92.19%] [G loss: 0.411267, mse: 0.405452]
8 [D loss: 0.300572, acc: 84.38%] [G loss: 0.353086, mse: 0.346715]
9 [D loss: 0.261502, acc: 95.31%] [G loss: 0.324826, mse: 0.318103]
10 [D loss: 0.233767, acc: 93.75%] [G loss: 0.314981, mse: 0.308377]
11 [D loss: 0.229087, acc: 95.31%] [G loss: 0.297036, mse: 0.290678]
12 [D loss: 0.207675, acc: 98.44%] [G loss: 0.309373, mse: 0.303373]
13 [D loss: 0.212236, acc: 98.44%] [G loss: 0.260016, mse: 0.253989]
14 [D loss: 0.196887, acc: 96.88%] [G loss: 0.278278, mse: 0.272521]
15 [D loss: 0.195900, acc: 98.44%] [G loss:

126 [D loss: 0.419991, acc: 78.12%] [G loss: 0.142379, mse: 0.137735]
127 [D loss: 0.280870, acc: 84.38%] [G loss: 0.148387, mse: 0.139508]
128 [D loss: 0.313424, acc: 79.69%] [G loss: 0.147364, mse: 0.141564]
129 [D loss: 0.304631, acc: 82.81%] [G loss: 0.140032, mse: 0.133939]
130 [D loss: 0.239193, acc: 90.62%] [G loss: 0.155615, mse: 0.147489]
131 [D loss: 0.311570, acc: 81.25%] [G loss: 0.133387, mse: 0.127254]
132 [D loss: 0.299675, acc: 92.19%] [G loss: 0.162579, mse: 0.158368]
133 [D loss: 0.205828, acc: 92.19%] [G loss: 0.147764, mse: 0.140676]
134 [D loss: 0.300950, acc: 89.06%] [G loss: 0.148851, mse: 0.143218]
135 [D loss: 0.243031, acc: 89.06%] [G loss: 0.159071, mse: 0.154510]
136 [D loss: 0.245902, acc: 89.06%] [G loss: 0.165500, mse: 0.159341]
137 [D loss: 0.222733, acc: 90.62%] [G loss: 0.154061, mse: 0.148528]
138 [D loss: 0.271293, acc: 87.50%] [G loss: 0.141032, mse: 0.136237]
139 [D loss: 0.253409, acc: 85.94%] [G loss: 0.144748, mse: 0.138875]
140 [D loss: 0.22474

...

19458 [D loss: 0.430583, acc: 79.69%] [G loss: 0.047733, mse: 0.045922]
19459 [D loss: 0.293737, acc: 89.06%] [G loss: 0.045089, mse: 0.043183]
19460 [D loss: 0.434700, acc: 81.25%] [G loss: 0.070227, mse: 0.068410]
19461 [D loss: 0.492791, acc: 81.25%] [G loss: 0.066857, mse: 0.064773]
19462 [D loss: 0.397612, acc: 81.25%] [G loss: 0.059511, mse: 0.057494]
19463 [D loss: 0.325684, acc: 84.38%] [G loss: 0.053543, mse: 0.051599]
19464 [D loss: 0.405833, acc: 90.62%] [G loss: 0.052871, mse: 0.050969]
19465 [D loss: 0.511307, acc: 76.56%] [G loss: 0.054651, mse: 0.052717]
19466 [D loss: 0.370201, acc: 79.69%] [G loss: 0.059356, mse: 0.057098]
19467 [D loss: 0.324627, acc: 85.94%] [G loss: 0.056842, mse: 0.054837]
19468 [D loss: 0.353993, acc: 85.94%] [G loss: 0.067112, mse: 0.065050]
19469 [D loss: 0.317160, acc: 82.81%] [G loss: 0.049568, mse: 0.047454]
19470 [D loss: 0.339648, acc: 89.06%] [G loss: 0.050434, mse: 0.048130]
19471 [D loss: 0.450647, acc: 79.69%] [G loss: 0.071313, mse: 0.

19572 [D loss: 0.542844, acc: 73.44%] [G loss: 0.057140, mse: 0.055093]
19573 [D loss: 0.438767, acc: 78.12%] [G loss: 0.061138, mse: 0.059208]
19574 [D loss: 0.300579, acc: 89.06%] [G loss: 0.057860, mse: 0.055701]
19575 [D loss: 0.343434, acc: 90.62%] [G loss: 0.055553, mse: 0.053570]
19576 [D loss: 0.311305, acc: 87.50%] [G loss: 0.049809, mse: 0.047627]
19577 [D loss: 0.473465, acc: 78.12%] [G loss: 0.063593, mse: 0.061946]
19578 [D loss: 0.246077, acc: 90.62%] [G loss: 0.052265, mse: 0.049982]
19579 [D loss: 0.429616, acc: 79.69%] [G loss: 0.053838, mse: 0.051838]
19580 [D loss: 0.473842, acc: 75.00%] [G loss: 0.065806, mse: 0.064138]
19581 [D loss: 0.367551, acc: 79.69%] [G loss: 0.060546, mse: 0.058927]
19582 [D loss: 0.359654, acc: 85.94%] [G loss: 0.055015, mse: 0.052479]
19583 [D loss: 0.345793, acc: 82.81%] [G loss: 0.056781, mse: 0.054854]
19584 [D loss: 0.387420, acc: 82.81%] [G loss: 0.052303, mse: 0.050139]
19585 [D loss: 0.388901, acc: 84.38%] [G loss: 0.057305, mse: 0.

19691 [D loss: 0.404280, acc: 81.25%] [G loss: 0.058602, mse: 0.056711]
19692 [D loss: 0.330947, acc: 85.94%] [G loss: 0.050203, mse: 0.048492]
19693 [D loss: 0.297831, acc: 85.94%] [G loss: 0.055548, mse: 0.053365]
19694 [D loss: 0.319560, acc: 87.50%] [G loss: 0.056000, mse: 0.053807]
19695 [D loss: 0.401585, acc: 81.25%] [G loss: 0.060477, mse: 0.058599]
19696 [D loss: 0.406510, acc: 84.38%] [G loss: 0.056314, mse: 0.054311]
19697 [D loss: 0.467584, acc: 81.25%] [G loss: 0.059233, mse: 0.057200]
19698 [D loss: 0.394110, acc: 79.69%] [G loss: 0.053948, mse: 0.051919]
19699 [D loss: 0.361143, acc: 84.38%] [G loss: 0.062300, mse: 0.060295]
19700 [D loss: 0.288371, acc: 87.50%] [G loss: 0.061852, mse: 0.059836]
19701 [D loss: 0.390291, acc: 84.38%] [G loss: 0.059710, mse: 0.057665]
19702 [D loss: 0.294706, acc: 89.06%] [G loss: 0.060729, mse: 0.058282]
19703 [D loss: 0.382004, acc: 85.94%] [G loss: 0.049226, mse: 0.047108]
19704 [D loss: 0.320960, acc: 90.62%] [G loss: 0.053649, mse: 0.

19810 [D loss: 0.362703, acc: 82.81%] [G loss: 0.052845, mse: 0.050995]
19811 [D loss: 0.498486, acc: 78.12%] [G loss: 0.061916, mse: 0.060282]
19812 [D loss: 0.467679, acc: 82.81%] [G loss: 0.054575, mse: 0.052760]
19813 [D loss: 0.377222, acc: 84.38%] [G loss: 0.054227, mse: 0.052208]
19814 [D loss: 0.344891, acc: 84.38%] [G loss: 0.056799, mse: 0.055037]
19815 [D loss: 0.521560, acc: 75.00%] [G loss: 0.060150, mse: 0.058189]
19816 [D loss: 0.352585, acc: 84.38%] [G loss: 0.051464, mse: 0.049490]
19817 [D loss: 0.298835, acc: 89.06%] [G loss: 0.055319, mse: 0.053225]
19818 [D loss: 0.360934, acc: 84.38%] [G loss: 0.053738, mse: 0.051870]
19819 [D loss: 0.468210, acc: 81.25%] [G loss: 0.052658, mse: 0.051052]
19820 [D loss: 0.345365, acc: 85.94%] [G loss: 0.064219, mse: 0.062219]
19821 [D loss: 0.463299, acc: 75.00%] [G loss: 0.053310, mse: 0.051562]
19822 [D loss: 0.409481, acc: 81.25%] [G loss: 0.052150, mse: 0.050121]
19823 [D loss: 0.398415, acc: 82.81%] [G loss: 0.056660, mse: 0.

19930 [D loss: 0.456960, acc: 82.81%] [G loss: 0.062385, mse: 0.060636]
19931 [D loss: 0.359939, acc: 82.81%] [G loss: 0.052970, mse: 0.051015]
19932 [D loss: 0.354724, acc: 84.38%] [G loss: 0.052248, mse: 0.050182]
19933 [D loss: 0.336688, acc: 85.94%] [G loss: 0.051064, mse: 0.049083]
19934 [D loss: 0.466662, acc: 76.56%] [G loss: 0.057506, mse: 0.055946]
19935 [D loss: 0.336336, acc: 84.38%] [G loss: 0.060634, mse: 0.058480]
19936 [D loss: 0.419833, acc: 81.25%] [G loss: 0.058333, mse: 0.056532]
19937 [D loss: 0.413643, acc: 84.38%] [G loss: 0.057055, mse: 0.055115]
19938 [D loss: 0.471954, acc: 79.69%] [G loss: 0.067134, mse: 0.065425]
19939 [D loss: 0.548711, acc: 79.69%] [G loss: 0.057187, mse: 0.055329]
19940 [D loss: 0.478830, acc: 73.44%] [G loss: 0.059649, mse: 0.058070]
19941 [D loss: 0.362452, acc: 87.50%] [G loss: 0.058455, mse: 0.056754]
19942 [D loss: 0.510347, acc: 78.12%] [G loss: 0.053226, mse: 0.051512]
19943 [D loss: 0.419093, acc: 82.81%] [G loss: 0.058295, mse: 0.